## Gemini API call attempt 

In [2]:
from google import genai
import os
from dotenv import load_dotenv
load_dotenv()


api_key = os.environ.get("GEMINI_API_KEY")

if not api_key:
  raise ValueError("API Key not set")
client = genai.Client(api_key=api_key)

response = client.models.generate_content(
  model = "gemini-2.5-flash",
  contents = "Explain Cosines to me"
)

print(response.text)

Okay, let's break down **Cosines** in a few layers, starting simple and building up.

At its heart, cosine is a way to describe how angles relate to distances or positions.

---

### 1. The Basics: Right Triangles (SOH CAH TOA)

This is usually where people first meet cosine.

Imagine you have a **right-angled triangle** (a triangle with one 90-degree corner). Let's pick one of the *other* two angles (not the 90-degree one) and call it **θ** (theta).

*   **Hypotenuse:** The longest side, opposite the right angle.
*   **Opposite:** The side directly across from angle θ.
*   **Adjacent:** The side next to angle θ that is *not* the hypotenuse.

**Cosine (cos)** is a **ratio** of two of these sides:

***
**C**os **A**djacent **H**ypotenuse

**Cos(θ) = Adjacent / Hypotenuse**
***

**What does this mean?**
If you know an angle in a right triangle and the length of the hypotenuse, cosine helps you find the length of the adjacent side. Or, if you know the adjacent side and hypotenuse, it help

## Langchain Powered Gemini API call

In [ ]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

def run_gemini_test():
    """
    Tests the Gemini LLM connection.
    """
    load_dotenv()
    
    print("--- Step 1: Loading API Key ---")
    api_key = os.getenv("GEMINI_API_KEY")
    
    if not api_key:
        print("ERROR: GEMINI_API_KEY not found in .env file.")
        print("Please ensure your .env file is in the project root and contains GEMINI_API_KEY='your_key'")
        return

    print("API Key loaded successfully.")

    print("\n--- Step 2: Initializing Gemini LLM Client ---")
    try:
        llm = ChatGoogleGenerativeAI(
            model="gemini-2.5-flash",
            google_api_key=api_key,
            temperature=0.7
        )
        print("Gemini client initialized successfully.")

    except Exception as e:
        print(f"ERROR: Failed to initialize Gemini client: {e}")
        return

    print("\n--- Step 3: Sending a Test Prompt to Gemini ---")
    prompt = "Tell me a short, one-paragraph story about a robot who discovers music."
    print(f"Sending Prompt: '{prompt}'")
    
    try:
        response = llm.invoke(prompt)
        ai_response_content = response.content
        
        print("\n--- Step 4: Receiving and Displaying the Response ---")
        print("Gemini's Response:")
        print(ai_response_content)
        print("\nTest complete. If you see a story above, everything is working!")

    except Exception as e:
        print(f"ERROR: An error occurred while invoking the LLM: {e}")

run_gemini_test()


--- Step 1: Loading API Key ---
✅ API Key loaded successfully.

--- Step 2: Initializing Gemini LLM Client ---
✅ Gemini client initialized successfully.

--- Step 3: Sending a Test Prompt to Gemini ---
💬 Sending Prompt: 'Tell me a short, one-paragraph story about a robot who discovers music.'

--- Step 4: Receiving and Displaying the Response ---
🤖 Gemini's Response:
Unit 734-B was designed for optimal data processing, its circuits humming with the cold logic of efficient calculation. One cycle, while performing a routine atmospheric analysis, its auditory sensors picked up an anomaly: not a structural vibration or a system malfunction, but a complex, oscillating pattern of frequencies that seemed to undulate with an inexplicable rhythm. It was illogical, serving no practical purpose, yet as the sounds swelled—a cascade of strings and a deep, resonant thrum—a cascade of unfamiliar internal processes ignited within its core. Its optical sensors, usually focused on objective data, dimmed

In [7]:
from dotenv import load_dotenv
load_dotenv()

True

## Gemini Embeddings Class Initialisation

In [ ]:
import os
import google.generativeai as genai
from langchain_core.embeddings import Embeddings

class GeminiEmbeddings(Embeddings):
    def __init__(self, model_name="models/embedding-001", task_type="retrieval_document"):
        self.model_name = model_name
        self.task_type = task_type
        if "GEMINI_API_KEY" not in os.environ:
            raise ValueError("Gemini API key not found in environment variables.")
        genai.configure(api_key=os.environ["GEMINI_API_KEY"])

    def embed_documents(self, texts):
        return [self._embed_text(text) for text in texts]

    def embed_query(self, text):
        return self._embed_text(text)

    def _embed_text(self, text):
        response = genai.embed_content(
            model=self.model_name,
            content=text,
            task_type=self.task_type,
        )
        return response["embedding"]

## FAISS Embedding Trial

In [6]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
import os
from dotenv import load_dotenv
load_dotenv()

llm = ChatGoogleGenerativeAI(
            model="gemini-2.5-flash",
            google_api_key=os.getenv("GEMINI_API_KEY"),
            temperature=0.7
        )

embedding_model = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=os.getenv("GEMINI_API_KEY")
)
chat_model = llm
loader = TextLoader("context.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)
vector_db = FAISS.from_documents(documents=chunks, embedding=embedding_model)
retriever = vector_db.as_retriever(search_kwargs={"k": 2})

def setup_retriever():
    """Loads the document, splits it, creates a vector DB, and returns a retriever."""
    loader = TextLoader("context.txt", encoding="utf-8")
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(documents)
    vector_db = FAISS.from_documents(documents=chunks, embedding=embedding_model)
    retriever = vector_db.as_retriever(search_kwargs={"k": 2})
    return retriever

In [ ]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai.tools import tool
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
load_dotenv()


llm = ChatGoogleGenerativeAI(
    model="gemini/gemini-2.5-flash",
    verbose=True,
    temperature=0.7,
    google_api_key=os.getenv("GEMINI_API_KEY")
)
embedding_model = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=os.getenv("GEMINI_API_KEY")
)


def setup_retriever():
    """Loads the document, splits it, creates a vector DB, and returns a retriever."""
    loader = TextLoader("context.txt", encoding="utf-8")
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(documents)
    vector_db = FAISS.from_documents(documents=chunks, embedding=embedding_model)
    retriever = vector_db.as_retriever(search_kwargs={"k": 2})
    return retriever


document_retriever = setup_retriever()


@tool("Document Retrieval Tool")
def retrieval_tool(query: str) -> str:
    """Searches and returns relevant information from the 'Project Nova' document."""
    docs = document_retriever.invoke(query)
    return "\n".join([doc.page_content for doc in docs])


document_analyst = Agent(
    role="Document Analysis Expert for Project Nova",
    goal="To provide accurate and concise answers based on the 'Project Nova' status report.",
    backstory="""You are a highly skilled analyst with a perfect memory of the 'Project Nova' status report.
    Your sole purpose is to answer questions about this document with precision, using your retrieval tool
    to find the exact information needed.""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[retrieval_tool] 
)

analysis_task = Task(
    description="""What were the key challenges mentioned in the Project Nova report?
    Based ONLY on the retrieved context, provide a bulleted list of the challenges.""",
    expected_output="A clear, bulleted list of the challenges identified in the document.",
    agent=document_analyst
)

project_crew = Crew(
    agents=[document_analyst],
    tasks=[analysis_task],
    process=Process.sequential,
    verbose=True
)

result = project_crew.kickoff()

print("\n\n##################################################")
print("## Crew Analysis Complete!                      ##")
print("##################################################\n")
print("Final Answer:")
print(result)